# ItaData2024
modal example

In [1]:
using Pkg
Pkg.activate(".")
# Pkg.resolve()
# Pkg.instantiate()
# Pkg.dev("SoleModels")
# Pkg.dev("Audio911")
# Pkg.dev("SoleData")
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random
using StatsBase, Catch22

  Activating project at `~/audio-rules2024`


### Open .jld2 file
the file contains 504 samples of respiratory sound, labeled with 2 classes: healty and pneumonia

In [2]:
ds_path = "/datasets/respiratory_Healthy_Pneumonia"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

### Audio features extraction function
This function is called for every audio sample and extract 51 features:
26 bands of the mel spectrogram,
13 coefficients of the mfcc
12 spectral features: centroid, crest, entropy, f0, flatness, flux, kurtosis, rolloff, skewness, decrease, slope, spread

In [3]:
nan_replacer!(x::AbstractArray{Float64}) = replace!(x, NaN => 0.0)

function afe(x::AbstractVector{Float64}; get_only_melfreq=false)
    # -------------------------------- parameters -------------------------------- #
    # audio module
    sr = 8000
    norm = true
    speech_detection = false
    # stft module
    stft_length = 1024
    win_type = (:hann, :periodic)
    win_length = 1024
    overlap_length = 512
    stft_norm = :power                      # :power, :magnitude, :pow2mag
    # mel filterbank module
    nbands = 26
    scale = :mel_htk                        # :mel_htk, :mel_slaney, :erb, :bark
    melfb_norm = :bandwidth                 # :bandwidth, :area, :none
    freq_range = (300, round(Int, sr / 2))
    # mel spectrogram module
    db_scale = false
    # mfcc module
    ncoeffs = 13
    rectification = :log                    # :log, :cubic_root
    dither = true
    # f0 module
    method = :nfc
    f0_range = (50, 400)

    # --------------------------------- functions -------------------------------- #
    # audio module
    audio = load_audio(
        file=x,
        sr=sr,
        norm=norm,
    );

    stftspec = get_stft(
        audio=audio,
        stft_length=stft_length,
        win_type=win_type,
        win_length=win_length,
        overlap_length=overlap_length,
        norm=stft_norm
    );

    # mel filterbank module
    melfb = get_melfb(
        stft=stftspec,
        nbands=nbands,
        scale=scale,
        norm=melfb_norm,
        freq_range=freq_range
    );

    if get_only_melfreq
        return melfb.freq
    end

    # mel spectrogram module
    melspec =  get_melspec(
        stft=stftspec,
        fbank=melfb,
        db_scale=db_scale
    );

    # mfcc module
    mfcc = get_mfcc(
        source=melspec,
        ncoeffs=ncoeffs,
        rectification=rectification,
        dither=dither,
    );

    # f0 module
    f0 = get_f0(
        source=stftspec,
        method=method,
        freq_range=f0_range
    );

    # spectral features module
    spect = get_spectrals(
        source=stftspec,
        freq_range=freq_range
    );

    x_features = hcat(
        melspec.spec',
        mfcc.mfcc',
        f0.f0,
        spect.centroid,
        spect.crest,
        spect.entropy,
        spect.flatness,
        spect.flux,
        spect.kurtosis,
        spect.rolloff,
        spect.skewness,
        spect.decrease,
        spect.slope,
        spect.spread
    );

    nan_replacer!(x_features)

    return x_features
end

afe (generic function with 1 method)

### Compute DataFrame of features

In [4]:
color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36)
freq = round.(Int, afe(x[1, :audio]; get_only_melfreq=true))
r_select = r"\e\[\d+m(.*?)\e\[0m"
variable_names = [
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:26]...,
    ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:13]...,
    "\e[$(color_code[:green])mf0\e[0m", "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
]

X = DataFrame([name => Vector{Float64}[] for name in [match(r_select, v)[1] for v in variable_names]])

function mean_longstretch1(x) Catch22.SB_BinaryStats_mean_longstretch1((x)) end
function diff_longstretch0(x) Catch22.SB_BinaryStats_diff_longstretch0((x)) end
function quantile_hh(x) Catch22.SB_MotifThree_quantile_hh((x)) end
function sumdiagcov(x) Catch22.SB_TransitionMatrix_3ac_sumdiagcov((x)) end

nan_guard = [:std, :mean_longstretch1, :diff_longstretch0, :quantile_hh, :sumdiagcov]

for f_name in nan_guard
    @eval (function $(Symbol(string(f_name)*"+"))(channel)
        val = $(f_name)(channel)

        if isnan(val)
            SoleData.aggregator_bottom(SoleData.existential_aggregator(≥), eltype(channel))
        else
            eltype(channel)(val)
        end
    end)
    @eval (function $(Symbol(string(f_name)*"-"))(channel)
        val = $(f_name)(channel)

        if isnan(val)
            SoleData.aggregator_bottom(SoleData.existential_aggregator(≤), eltype(channel))
        else
            eltype(channel)(val)
        end
    end)
end

function get_patched_feature(f::Base.Callable, polarity::Symbol)
    if f in [minimum, maximum, StatsBase.mean, median]
        f
    else
        @eval $(Symbol(string(f)*string(polarity)))
    end
end

features = [
    (≥, get_patched_feature(maximum, :+)),            (≤, get_patched_feature(maximum, :-)),
    (≥, get_patched_feature(minimum, :+)),            (≤, get_patched_feature(minimum, :-)),
    (≥, get_patched_feature(StatsBase.mean, :+)),     (≤, get_patched_feature(StatsBase.mean, :-)),
    (≥, get_patched_feature(median, :+)),             (≤, get_patched_feature(median, :-)),
	(≥, get_patched_feature(std, :+)),                (≤, get_patched_feature(std, :-)),
	(≥, get_patched_feature(mean_longstretch1, :+)),  (≤, get_patched_feature(mean_longstretch1, :-)),
	(≥, get_patched_feature(diff_longstretch0, :+)),  (≤, get_patched_feature(diff_longstretch0, :-)),
	(≥, get_patched_feature(quantile_hh, :+)),        (≤, get_patched_feature(quantile_hh, :-)),
	(≥, get_patched_feature(sumdiagcov, :+)),         (≤, get_patched_feature(sumdiagcov, :-)),
]

for i in 1:nrow(x)
    push!(X, collect(eachcol(afe(x[i, :audio]))))
end

yc = CategoricalArray(y);

### Data compression for modal analysis

In [5]:
train_ratio = 0.8
train, test = partition(eachindex(yc), train_ratio, shuffle=true)
# train, test = partition(eachindex(yc), train_ratio, shuffle=false) ### Debug
X_train, y_train = X[train, :], yc[train]
X_test, y_test = X[test, :], yc[test]

println("Training set size: ", size(X_train), " - ", length(y_train))
println("Test set size: ", size(X_test), " - ", length(y_test))

Training set size: (403, 51) - 403
Test set size: (101, 51) - 101


### Train a model

In [6]:
learned_dt_tree = begin
    model = ModalDecisionTree(; relations = :IA7, features = features)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
    mach = machine(model, X_train, y_train) |> fit!
end

report(learned_dt_tree).printmodel(variable_names_map=variable_names);

┌ Info: Precomputing logiset...
└ @ ModalDecisionTrees.MLJInterface /home/paso/.julia/dev/ModalDecisionTrees.jl/src/interfaces/MLJ/wrapdataset.jl:135
┌ Info: Training machine(ModalDecisionTree(max_depth = nothing, …), …).
└ @ MLJBase /home/paso/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


▣ ⟨G⟩(max[mfcc3] ≤ 0.08932931340872206)
├✔ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨AO⟩(min[mfcc2] ≤ 2.3026736620518395))
│├✔ Healthy
│└✘ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨G⟩(max[mfcc9] ≥ 0.6113784530484502))
│ ├✔ Healthy
│ └✘ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨D̅B̅E̅⟩(mean[mfcc7] ≤ -0.8141953400588126))
│  ├✔ Healthy
│  └✘ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨D̅B̅E̅⟩(mean[mfcc13] ≤ -0.22491697821633488))
│   ├✔ Healthy
│   └✘ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨A̅O̅⟩(min[mfcc2] ≥ 5.737841125784729))
│    ├✔ Pneumonia
│    └✘ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨AO⟩(min[mfcc3] ≤ 0.04225118548264824))
│     ├✔ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨AO⟩((min[mfcc3] ≤ 0.04225118548264824) ∧ ⟨A̅O̅⟩(max[decrs] ≤ -0.8159022665280041)))
│     │├✔ Pneumonia
│     │└✘ Healthy
│     └✘ Pneumonia
└✘ ⟨G⟩(quantile_hh+[kurts] ≥ 2.1458417525947544)
 ├✔ ⟨G⟩((quantile_hh+[kurts] ≥ 2.1458417525947544) ∧ ⟨D̅B̅E̅⟩(sumdiagcov-[crest] ≤ 0.005509641873278238))
 │├✔ ⟨G⟩((qua

### Model inspection & rule study

In [7]:
_, mtree = report(mach).sprinkle(X_test, y_test)
sole_dt = ModalDecisionTrees.translate(mtree)

printmodel(sole_dt; show_metrics = true, variable_names_map=variable_names);

Applying tree... 100%|███████████████████████████████████| Time: 0:00:02


▣ ⟨G⟩(max[mfcc3] ≤ 0.08932931340872206)
├✔ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨AO⟩(min[mfcc2] ≤ 2.3026736620518395))
│├✔ Healthy : (ninstances = 25, ncovered = 25, confidence = 0.96, lift = 1.0)
│└✘ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨G⟩(max[mfcc9] ≥ 0.6113784530484502))
│ ├✔ Healthy : (ninstances = 5, ncovered = 5, confidence = 1.0, lift = 1.0)
│ └✘ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨D̅B̅E̅⟩(mean[mfcc7] ≤ -0.8141953400588126))
│  ├✔ Healthy : (ninstances = 3, ncovered = 3, confidence = 1.0, lift = 1.0)
│  └✘ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨D̅B̅E̅⟩(mean[mfcc13] ≤ -0.22491697821633488))
│   ├✔ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨D̅B̅E̅⟩((mean[mfcc13] ≤ -0.22491697821633488) ∧ (min[mel12=1289Hz] ≥ 0.00021809529872277814)))
│   │├✔ Healthy : (ninstances = 4, ncovered = 4, confidence = 0.75, lift = 1.0)
│   │└✘ Healthy : (ninstances = 1, ncovered = 1, confidence = 0.0, lift = NaN)
│   └✘ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨A̅O̅⟩(min[mfcc2] ≥ 5.73784

### Extract rules that are at least as good as a random baseline model

In [8]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0);
printmodel.(interesting_rules; show_metrics = true, variable_names_map=variable_names);

▣ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨AO⟩(min[mfcc2] ≤ 2.3026736620518395))  ↣  Healthy : (ninstances = 101, ncovered = 25, coverage = 0.25, confidence = 0.96, lift = 2.31, natoms = 2)
▣ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨G⟩(max[mfcc9] ≥ 0.6113784530484502)) ∧ [G]((max[mfcc3] ≤ 0.08932931340872206) → [AO](min[mfcc2] > 2.3026736620518395))  ↣  Healthy : (ninstances = 101, ncovered = 5, coverage = 0.05, confidence = 1.0, lift = 2.4, natoms = 4)
▣ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨D̅B̅E̅⟩(mean[mfcc7] ≤ -0.8141953400588126)) ∧ [G]((max[mfcc3] ≤ 0.08932931340872206) → [AO](min[mfcc2] > 2.3026736620518395)) ∧ [G]((max[mfcc3] ≤ 0.08932931340872206) → [G](max[mfcc9] < 0.6113784530484502))  ↣  Healthy : (ninstances = 101, ncovered = 3, coverage = 0.03, confidence = 1.0, lift = 2.4, natoms = 6)
▣ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨D̅B̅E̅⟩((mean[mfcc13] ≤ -0.22491697821633488) ∧ (min[mel12=1289Hz] ≥ 0.00021809529872277814))) ∧ [G]((max[mfcc3] ≤ 0.08932931340872206) → [

### Simplify rules while extracting and prettify result

In [9]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0, normalize = true);
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map=variable_names);

▣ ⟨G⟩((max[mfcc3] ≤ 0.09) ∧ ⟨AO⟩(min[mfcc2] ≤ 2.3))  ↣  Healthy : (ninstances = 101, ncovered = 25, coverage = 0.25, confidence = 0.96, lift = 2.31, natoms = 2)
▣ ⟨G⟩((max[mfcc3] ≤ 0.09) ∧ ⟨G⟩(max[mfcc9] ≥ 0.61)) ∧ [G]((max[mfcc3] ≤ 0.09) → [AO](min[mfcc2] > 2.3))  ↣  Healthy : (ninstances = 101, ncovered = 5, coverage = 0.05, confidence = 1.0, lift = 2.4, natoms = 4)
▣ ⟨G⟩((max[mfcc3] ≤ 0.09) ∧ ⟨D̅B̅E̅⟩(mean[mfcc7] ≤ -0.81)) ∧ [G]((max[mfcc3] ≤ 0.09) → [AO](min[mfcc2] > 2.3)) ∧ [G]((max[mfcc3] ≤ 0.09) → [G](max[mfcc9] < 0.61))  ↣  Healthy : (ninstances = 101, ncovered = 3, coverage = 0.03, confidence = 1.0, lift = 2.4, natoms = 6)
▣ ⟨G⟩((max[mfcc3] ≤ 0.09) ∧ ⟨D̅B̅E̅⟩((mean[mfcc13] ≤ -0.22) ∧ (min[mel12=1289Hz] ≥ 0.0))) ∧ [G]((max[mfcc3] ≤ 0.09) → [AO](min[mfcc2] > 2.3)) ∧ [G]((max[mfcc3] ≤ 0.09) → [G](max[mfcc9] < 0.61)) ∧ [G]((max[mfcc3] ≤ 0.09) → [D̅B̅E̅](mean[mfcc7] > -0.81))  ↣  Healthy : (ninstances = 101, ncovered = 4, coverage = 0.04, confidence = 0.75, lift = 1.8, natoms = 9)


### Directly access rule metrics

In [10]:
readmetrics.(listrules(sole_dt; min_lift=1.0, min_ninstances = 0))

10-element Vector{@NamedTuple{ninstances::Int64, ncovered::Int64, coverage::Float64, confidence::Float64, lift::Float64, natoms::Int64}}:
 (ninstances = 101, ncovered = 25, coverage = 0.24752475247524752, confidence = 0.96, lift = 2.3085714285714287, natoms = 2)
 (ninstances = 101, ncovered = 5, coverage = 0.04950495049504951, confidence = 1.0, lift = 2.4047619047619047, natoms = 4)
 (ninstances = 101, ncovered = 3, coverage = 0.0297029702970297, confidence = 1.0, lift = 2.4047619047619047, natoms = 6)
 (ninstances = 101, ncovered = 4, coverage = 0.039603960396039604, confidence = 0.75, lift = 1.8035714285714286, natoms = 9)
 (ninstances = 101, ncovered = 4, coverage = 0.039603960396039604, confidence = 1.0, lift = 1.7118644067796611, natoms = 10)
 (ninstances = 101, ncovered = 2, coverage = 0.019801980198019802, confidence = 1.0, lift = 1.7118644067796611, natoms = 13)
 (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 2.4047619047619047, natom

### Show rules with an additional metric (syntax height of the rule's antecedent)

In [11]:
printmodel.(sort(interesting_rules, by = readmetrics); show_metrics = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), variable_names_map=variable_names);

▣ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨AO⟩((min[mfcc3] ≤ 0.04225118548264824) ∧ (max[mel1=359Hz] ≤ 0.00252905104637584))) ∧ [G]((max[mfcc3] ≤ 0.08932931340872206) → [AO](min[mfcc2] > 2.3026736620518395)) ∧ [G]((max[mfcc3] ≤ 0.08932931340872206) → [G](max[mfcc9] < 0.6113784530484502)) ∧ [G]((max[mfcc3] ≤ 0.08932931340872206) → [D̅B̅E̅](mean[mfcc7] > -0.8141953400588126)) ∧ [G]((max[mfcc3] ≤ 0.08932931340872206) → [D̅B̅E̅](mean[mfcc13] > -0.22491697821633488)) ∧ [G]((max[mfcc3] ≤ 0.08932931340872206) → [A̅O̅](min[mfcc2] < 5.737841125784729)) ∧ [G]((max[mfcc3] ≤ 0.08932931340872206) → [AO]((min[mfcc3] ≤ 0.04225118548264824) → [A̅O̅](max[decrs] > -0.8159022665280041)))  ↣  Healthy : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 2.4047619047619047, natoms = 16, height = 10)
▣ ⟨G⟩((max[mfcc3] ≤ 0.08932931340872206) ∧ ⟨AO⟩((min[mfcc3] ≤ 0.04225118548264824) ∧ ⟨A̅O̅⟩(max[decrs] ≤ -0.8159022665280041))) ∧ [G]((max[mfcc3] ≤ 0.08932931340872206) →

### Pretty table of rules and their metrics

In [12]:
metricstable(interesting_rules; variable_names_map=variable_names, metrics_kwargs = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))))

┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬────────────┬──────────┬────────────┬────────────┬─────────┬────────┬────────┐
│                                                                                                                                                                                                                                                                             